In [1]:
# Importamos las librerías que necesitamos
import numpy as np
# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import re
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', 1000)          # Set the width to accommodate more columns
pd.set_option('display.max_colwidth', None)   # Set maximum column width to None (auto adjust)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
url_campeonatos = f'https://www.basketball-reference.com/teams/'

response = requests.get(url_campeonatos)

if str(response) == '<Response [200]>':
    print(response.status_code)
    sopa = BeautifulSoup(response.content, 'html.parser')
else:
    print('Hay un error')

200


In [25]:
tabla = sopa.find("div", id="div_teams_active").find("table")
filas = tabla.find('tbody').find_all('tr', class_='full_table') # la lista muestra todos los equipos históricos pero me quedé solo con aquellos que estan presentes ahora, y en la época de Jordan
team_data = [] # lista vacia para ir colocando todo mi diccionario

for i in filas: #itera sobre cada fila
    # Ignorar filas que sean comentarios o separadores
    if i.get('class') == ['thead']:
        continue
    
    # Obtener el nombre de la franquicia desde el primer 'th'
    franquicia = i.find("th").text.strip()
    
    # Obtener los demás valores desde 'td'
    valores = i.find_all("td")
    
        # Asegúrate de que no haya filas vacías
    
    liga = valores[0].text.strip()
    desde = valores[1].text.strip()
    hasta = valores[2].text.strip()
    anios = valores[3].text.strip()
    partidos = valores[4].text.strip()
    victorias = valores[5].text.strip()
    derrotas = valores[6].text.strip()
    porcentage_vic = valores[7].text.strip()
    playoffs = valores[8].text.strip()
    div_titulos = valores[9].text.strip()
    conf_titulos = valores[10].text.strip()
    campeonatos = valores[11].text.strip()
    
    # Agregar los datos a la lista principal
    team_data.append([
        franquicia, liga, desde, hasta, anios, partidos, victorias, 
        derrotas, porcentage_vic, playoffs, div_titulos, conf_titulos, campeonatos
    ])

# Convertir la lista de listas a un DataFrame de pandas
df = pd.DataFrame(team_data, columns=[
    "Franquicia", "Liga", "Desde", "Hasta", "Anios", "G", "W", "L", "W/L%", 
    "Plyfs", "Div", "Conf", "Champ"
])

In [26]:
len(df['Franquicia'].unique())

30

In [32]:
team_codes = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS',
}


In [33]:
len(team_codes)

30

In [34]:
df['team_code'] = df['Franquicia'].map(team_codes)


In [35]:
df.head(1)

,Franquicia,team_code,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,ATL,NBA,1949-50,2024-25,76,5943,2929,3014,.493,49,12,0,1


In [36]:
columnas = df.columns.tolist()
columnas.insert(1, columnas.pop(columnas.index('team_code')))

df = df[columnas]

In [37]:
df[df['team_code'] == 'LAL']

,Franquicia,team_code,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ
13,Los Angeles Lakers,LAL,NBA/BAA,1948-49,2024-25,77,6010,3554,2456,.591,64,33,19,17


In [38]:
df.head()

,Franquicia,team_code,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,ATL,NBA,1949-50,2024-25,76,5943,2929,3014,.493,49,12,0,1
1,Boston Celtics,BOS,NBA/BAA,1946-47,2024-25,79,6120,3639,2481,.595,61,34,11,18
2,Brooklyn Nets,BKN,NBA/ABA,1967-68,2024-25,58,4618,2031,2587,.440,31,5,2,2
3,Charlotte Hornets,CHA,NBA,1988-89,2024-25,35,2718,1176,1542,.433,10,0,0,0
4,Chicago Bulls,CHI,NBA,1966-67,2024-25,59,4686,2386,2300,.509,36,9,6,6


In [40]:
df.to_csv('../datos/franquicias_historicas.csv', index=False)

In [41]:
df

,Franquicia,team_code,Liga,Desde,Hasta,Anios,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,ATL,NBA,1949-50,2024-25,76,5943,2929,3014,.493,49,12,0,1
1,Boston Celtics,BOS,NBA/BAA,1946-47,2024-25,79,6120,3639,2481,.595,61,34,11,18
2,Brooklyn Nets,BKN,NBA/ABA,1967-68,2024-25,58,4618,2031,2587,.440,31,5,2,2
3,Charlotte Hornets,CHA,NBA,1988-89,2024-25,35,2718,1176,1542,.433,10,0,0,0
4,Chicago Bulls,CHI,NBA,1966-67,2024-25,59,4686,2386,2300,.509,36,9,6,6
5,Cleveland Cavaliers,CLE,NBA,1970-71,2024-25,55,4359,2038,2321,.468,24,7,5,1
6,Dallas Mavericks,DAL,NBA,1980-81,2024-25,45,3548,1800,1748,.507,25,5,3,1
7,Denver Nuggets,DEN,NBA/ABA,1967-68,2024-25,58,4618,2369,2249,.513,39,13,1,1
8,Detroit Pistons,DET,NBA/BAA,1948-49,2024-25,77,6004,2828,3176,.471,42,11,5,3
9,Golden State Warriors,GSW,NBA/BAA,1946-47,2024-25,79,6108,2973,3135,.487,37,12,7,7
